# Plotting


## 0. Import Required Software

In [1]:
from ipyfilechooser import FileChooser
from pathlib import Path
import sys

from mintpy.cli import view, tsview, plot_network, plot_transection, plot_coherence_matrix
import mintpy.plot_coherence_matrix
import mintpy.objects.insar_vs_gps
import mintpy.utils

current = Path("..").resolve()
sys.path.append(str(current))
import util.gps as gps

---
## 1. Select your project's custom config file

- This is located in your project's `MintPy` directory
- It is a text file named after your project
  - `path/to/MinPy/my_project.txt`

In [ ]:
path = Path.home()
fc = FileChooser(path)
print("Select your project's MintPy directory:")
display(fc)

In [ ]:
config_path = Path(fc.selected)
mint_path = config_path.parent
plot_path = mint_path / "plots"

---
## 2. Plot the interferogram network

Running **plot_network.py** gives an overview of the network and the average coherence of the stack. The program creates multiple files as follows:
- `ifgramStack_coherence_spatialAvg.txt`: Contains interferogram dates, average coherence temporal and spatial baseline separation.
- `Network.pdf`: Displays the network of interferograms on time-baseline coordinates, colorcoded by avergae coherence of the interferograms. 
- `CoherenceMatrix.pdf` shows the avergae coherence pairs between all available pairs in the stack.

In [ ]:
%matplotlib inline
with osl.work_dir(mint_path):
    scp_args = f'{inputs_path}/ifgramStack.h5 --cmap-vlist 0.2 0.7 1.0'
    plot_network.main(scp_args.split())

**Plot the velocity map**

Note: The min/max values in this plot should be adjusted to your data set. 

The output of the following call will show a data range. You can run the cell once to collect this information and re-run it after adjustments are made.

- You should update `vmin` and `vmax` to match your data range
- The `vmin` and `vmax` values that you assign will be used for additional plots in this notebook
- If you wish to set 0 deformation to the center of the color scale (green), adjust your min/max values so they are symmetrical
    - For example, given a data range of `[-10.691118, 13.904866]`, appropriate symmetrical min/max values would be `[-14.0, 14.0]` 

In [ ]:
colormap_select = osl.select_parameter(
    [
        "Display red for uplift, and blue for subsidence",
        "Display blue for uplift, and red for subsidence"
    ]
)
display(colormap_select)

In [ ]:
%matplotlib inline
cmap = 'jet' if 'red for uplift' in colormap_select.value else 'jet_r'

vel_vmin = -8
vel_vmax = 8

scp_args = f'{mint_path}/velocity.h5 velocity -v {vel_vmin} {vel_vmax} --dpi 600 --figsize 15 15 --outfile {plot_path}/velocity.png --update --colormap {cmap}'
view.main(scp_args.split())

---
## 3. Plot the unwrapped inverted timeseries

In [ ]:
geotiff_path = mint_path / 'GeoTiffs'
disp_path = geotiff_path / 'displacement_maps'
unwrapped_path = disp_path / 'unwrapped'
unwrapped_path.mkdir(parents=True, exist_ok=True)
ts_demErr_path = list(mint_path.glob('timeseries*_demErr.h5'))[0]

In [ ]:
inv_vmin = -9
inv_vmax = 9

scp_args = f'{ts_demErr_path} --notitle --notick --noaxis -v {inv_vmin} {inv_vmax} --dpi 600 --figsize 15 15 --outfile {unwrapped_path}/unwrapped_inverted_ts.png'
view.main(scp_args.split())

---
## 4. Plot the Cumulative Displacement Map and Point Displacement Time Series

- Use the `Time` bar below the Cumulative Displacement Map to view displacements for different time periods
- Click on the Cumulative Displacement Map to select points for displaying Point Displacement Time-Series

In [ ]:
%matplotlib widget

tsview.main([str(ts_demErr_path),
             f'-d={mint_path}/inputs/geometryGeo.h5', 
             f'-o={mint_path}/displacement_ts', 
             f'--outfile={mint_path}/displacement_ts.pdf'])